# LArIAT - 2D Data Examples / Hand Scan Tutorial
## Training the Deep Neural Network in your head

This notebook introduces the 2D LArIAT data set and leads you through setting up a handscan. The included exercises are meant to introduce basic python, numpy, and h5 file/data structure manipulation, ploting in matplotlib, and the concepts of problem formulation, training, and validation.

## Introduction
Before High Energy Physicists used computers with automatic reconstruction to turn raw data into features, they relied on hand scans performed by people. In this notebook we will setup a hand scan using the Liquid Argon TPC (LArTPC) data we looked at last time. The task will be to identify the type of particle. You will be the handscanner. The steps are as follows:

    * Data Engineering: Load data from various files
    * Training: Train the handscanner by presenting images of the data with the labels.
    * Validation: Ask the handscanner to classify some randomly selected images, and see how well they do.


## Data Engineering

Our data is stored in a bunch of files. You can see the files by listing the directory using the unix "ls" command. You can call shell commands, like "ls", from Jupyter:

In [ ]:
BaseDir2D="/data/LArIAT/h5_files_2D_3D/2D_h5/*"
%ls $BaseDir2D

That's a lot of files. Lets count how many... in python. There are a variety of ways of getting back a directory listing in python. Here's one:

In [ ]:
import glob
Files=glob.glob(BaseDir2D)
print "Number of Files:", len(Files)
print "First Filename:", Files[0]

Looking at the file names, you notice that they start with the type of particle. Each file contains a samples of "events". In each event, we simulated shooting a particle into the detector and stored the response. The name of the file specifies what type of particle was simulated in that file.

Let's try to figure out what types. We'll loop over the file names, strip out the first part of the file name, and store it in a dictionary:

In [ ]:
import os

FileCount= {}  # Store the count here
FileLists= {}  # Organize the files by particle type here.

for aFile in Files:
    # Lets strip the path (everything before the "/"s) and get the filename:
    FileName=os.path.basename(aFile)
    
    # Now use everything before the first "_" as the particle name
    ParticleName=FileName.split('_')[0]
    
    if ParticleName in FileCount.keys():
        FileCount[ParticleName]+=1
        FileLists[ParticleName].append(aFile)
    else:
        FileCount[ParticleName]=1
        FileLists[ParticleName]= [aFile]
    
print "Number of types of particles:", len(FileCount.keys())
print "----------------------------------------------------------"
print "Number of files for each particle type:", FileCount
print "----------------------------------------------------------"
print "First file of each type:"
for aFile in FileLists:
    print aFile,":",FileLists[aFile][0]

We can count how many examples are in each file by open them up in h5py:

In [ ]:
import h5py

f=h5py.File(FileLists["electron"][0],"r")

# Read the First N_Events. Data is stored as float16, lets store it as float32 to avoid overflows later when we sum.
print "Shape of the data:", f["images"].shape
print "Number of events in file:", f["images"].shape[0]

f.close()

In [ ]:
f.keys()

## Training

We will use matplotlib for most of our plotting. There are lots of tutorials and primers out there that you can find searching the web. A good tutorial can be found in the [Scipy Lectures](http://www.scipy-lectures.org/intro/matplotlib/matplotlib.html). Look through these on your own time, it is not necessary for doing these exercise.

The raw data from a LArTPC detector looks like an image. The LArIAT detector, which we have simulated, has 2 readout views. The following code gives you an example how to plot these images. 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# Load the first electron file
f=h5py.File(FileLists["electron"][0],"r")

# Get the images
images=f["images"]

print "Data shape:", images.shape

def PlotEvent(image):
    # Make two plots. Create a 1 by 2 grid the plots.
    ax1 = plt.subplot(1,2,1)
    ax2 = plt.subplot(1,2,2)

    # Plot the first view. Note: [EventNumber, View] = [0,0]
    ax1.imshow(image[0])

    # Plot the second view 
    ax2.imshow(image[1])

    # The data is 240 by 4096. Change the aspect ratio so the plot is not squished. 
    ax1.set_aspect(16)  
    ax2.set_aspect(16) 

# Plot the 5th Event
PlotEvent(np.array(images[4],dtype="float32"))

f.close()

### Exercise 3.2.1- Setup Training

Write a function that takes a file, and creates a grid of plots showing the first N events. Use this function to plot the first 9 events in the first file of each particle type in a 3 by 3 grid. You only need to show one view.

In [ ]:
def PlotEvents(FileName, N_Events):
    ### BEGIN SOLUTION

    # Fill in your solution here        
    
    ### END SOLUTION
    pass

N_Events=9

for aFile in FileLists:
    FileName=FileLists[aFile][0]
    ParticleName=os.path.basename(FileName).split('_')[0]
    
    print ParticleName,":"
    PlotEvents(FileName,N_Events)

### Exercise 3.2.2- Train Yourself

By looking closely at each particle type, identify at least one "feature" that would allow you to by eye uniquely identify that particle type. 

Type you answer in this box.

### BEGIN SOLUTION

- muon/antimuon: your description here
- electron/antielectron: your description here
- pion: your description here
- pionPlus/pionMinus: your description here
- kaonPlus/kaonMinus: your description here
- photon: your description here
- nue/nuebar: your description here
- numu/numubar: your description here
- proton/antiproton: your description here

### END SOLUTION

## Validation

Now we have to setup a validation process. We will first assign each particle type a unique index. Then we will load some events of each particle type, mix them while keeping track of the indecies. Finally we will present the images to the handscanner, ask them to classify, and keep track of how well they do.

Read through and try to understand the following code which setups up 2 dictionaries we will use to uniquely identify particle types. 

In [ ]:
import numpy as np

# Assign index to particle type
ParticleTypesIndexMap = {}

for i,ParticleType in enumerate(FileLists.keys()):    
    ParticleTypesIndexMap[ParticleType]=i
    
# Merge particle/anti-particle
for ParticleName in ParticleTypesIndexMap:
    if 'bar' in ParticleName:
        try:
            ParticleicleTypesIndexMap[ParticleName]=ParticleTypesIndexMap[ParticleName.split('bar')[0]]
        except:
           pass
    
    if 'anti' in ParticleName:
        try:
            ParticleTypesIndexMap[ParticleName]=ParticleTypesIndexMap[ParticleName.split('anti')[1]]
        except:
            pass
        
    if 'Minus' in ParticleName:
        try:
            ParticleTypesIndexMap[ParticleName]=ParticleTypesIndexMap[ParticleName.split('Minus')[0]+"Plus"]
        except:
            pass
    
print "Index map:"
print ParticleTypesIndexMap

# Reverse Map
ParticleTypesIndexMapR={}

for p in ParticleTypesIndexMap:
    if ParticleTypesIndexMap[p] not in ParticleTypesIndexMapR:
        ParticleTypesIndexMapR[ParticleTypesIndexMap[p]]=p

print "Reverse Index map:"
print ParticleTypesIndexMapR



Now we load the data and mix them:

In [ ]:
Data_X = None
Data_Y = None
N_Events_perType=10

for ParticleType in FileLists:
    # Open the first file
    FileName=FileLists[ParticleType][1] # we will take the 2nd file so we don't use the training sample for validation
    print "Opening:",FileName
    f=h5py.File(FileName,"r")
    
    # Get the images/features
    images=np.array(f["images"][:N_Events_perType])
    
    # Warn if not enough events
    N_Events_read=images.shape[0]
    if not N_Events_read==N_Events_perType:
        print "Warning: Sample", FileName, "had only",N_Events_read,"events."
    
    # Assign labels
    labels=np.empty(N_Events_read)
    labels.fill(ParticleTypesIndexMap[ParticleType])

    # Store some of them
    try:
        # If we have already read some data, add to it
        Data_X=np.concatenate((Data_X,np.array(images,dtype="float32")))
        Data_Y=np.concatenate((Data_Y,np.array(labels,dtype="float32")))
    except:
        # If we haven't read any data yet
        Data_X=images
        Data_Y=labels
    
        
    f.close()

print Data_X.shape, Data_Y.shape

def shuffle_in_unison_inplace(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]
    
Data_X,Data_Y=shuffle_in_unison_inplace(Data_X,Data_Y)    


##  Exercise 3.3.1

The following code presents images and asks the handscanner for a type. Read through it carefully. Try it out. Then instrument this code so it keeps track of success and failures. The goal is to create a confusion matrix, a table that keeps track of how often each particle type is correctly identified and how often it is misidentified as any other type.  

In [ ]:
View=0

for image in Data_X:
    PlotEvent(image)
    plt.show()
    
    print "Select Type from:", ParticleTypesIndexMapR
    try:
        answer=int(raw_input('Input:'))
    except ValueError:
        print "Not a number"
        
    # Stop loop
    if answer==-1:
        break
    
    print "You selected:", ParticleTypesIndexMapR[answer]
        

##  Exercise 3.3.2

Make yourself the handscanner. Use above code to go through the full data sample and create a confusion matrix.

## Exercise 3.4.1
Write a function that downsamples all of images by summing samples to reduce the 4096 long dimension of the data.

## Exercise 3.4.2
Write a function that returns a sub-region in the 4096 long dimention where the total charge is max.